In [1]:
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from spafe.features.rplp import plp
import sounddevice as sd
from scipy.io.wavfile import write

In [ ]:
tdf_files = [
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ahmed_Armaghan\\SP80644_M_SNG_L56.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ahmed_Armaghan\\SP80644_M_SNG_L70.tdf',     
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L70.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L57.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L65.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L66.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L18.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L70.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L56.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L51.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L54.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L70.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L51.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L55.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L70.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L51.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L54.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L70.tdf',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L6.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L67.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L69.tdf'
]

audio_files = [
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ahmed_Armaghan\\SP80644_M_SNG_L56.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ahmed_Armaghan\\SP80644_M_SNG_L70.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L70.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L57.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L65.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L66.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L18.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L70.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L56.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L51.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L54.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L70.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L51.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L55.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L70.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L51.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L54.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L70.wav',
'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L6.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L67.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L69.wav'
]

total_time = 0
data_details = {}

for i in range(len(tdf_files)):
    with open(tdf_files[i], 'r') as f:
        lines = f.readlines()
    speaker_data = 0
    for line in lines[3:]:  # Skip the first three lines
        data = line.strip().split('\t')
        audio_file, channel, start, end, speaker, speakerType, speakerDialect, _, _, segment, _ = data
        segment = float(end) - float(start)
        if segment < 3:
            continue
        speaker_data += segment
    if speaker not in data_details:
        data_details[speaker] = round(speaker_data/60, 2)  # Initialize time for new speaker
    else:
        data_details[speaker] += round(speaker_data/60, 2)  # Update speaker's total time
    total_time += speaker_data

print('Total Time Available for Processing is ' + str(round(total_time/60, 2)) + ' minutes')
print('Individual Details of Speaker')
print(data_details)    

In [2]:
# Load MFCC features and labels from numpy files
MFCC = np.load('MFCC_features.npy')
PLP = np.load('PLP_features.npy')
labels = np.load('labels.npy')

In [6]:
sam_r = 8000;      # Set sample rate as 8kHz
classes = {"ali":1,"ayesha":2,"mahnoor":3,"maryam":4,"usman_amin":5,"askari":6,"usman_hassan":7,"huzaifa":8}


tdf_files = [
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L70.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L57.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L65.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L66.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L70.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L56.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L54.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L70.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L51.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L55.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L55.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L70.tdf',
    'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L51.tdf', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L54.tdf', 
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L6.tdf'#, 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L67.tdf'
            ]

audio_files = [
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L70.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ali_Hussain\\SP80643_M_SNG_L57.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L65.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Ayesha_Farooq\\SP80538_F_SNG_L66.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L70.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Mahnoor_Khalique\\SP80645_F_SNG_L56.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L54.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Maryam_Zahra\\SP80640_F_SNG_L70.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L51.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Usman_Amin\\SP80540_M_SNG_L55.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L55.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Askari_Zaidi\\SP80641_M_SNG_L70.wav',
    'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L51.wav', 'C:\\Users\\ApriZon\\Desktop\\CEP\Data\\Usman_Hassan\\SP80642_M_SNG_L54.wav', 
    'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L6.wav'#, 'C:\\Users\\ApriZon\\Desktop\\CEP\\Data\\Muhammad_Huzaifa\\SP80539_M_SNG_L67.wav'
              ]

In [ ]:
MFCC = []          # empty list to store extracted MFCC features of data
PLP = []           # empty list to store extracted PLP features of data
labels = []        # empty list to store labels of data

for i in range(len(tdf_files)):
    with open(tdf_files[i], 'r') as f:
        lines = f.readlines()

    for line in lines[3:]:  # Skip the first three lines
        data = line.strip().split('\t')
        audio_file, channel, start, end, speaker, speakerType, speakerDialect, _, _, segment, _ = data

        if (float(end) - float(start)) < 3:
            continue

        s = int(float(start) * sam_r)  # convert start time to samples
        e = int(float(end) * sam_r)  # convert end time to samples

        if not os.path.exists(audio_files[i]):
            print(f"Audio file not found: {audio_files[i]}")
            continue
        
        # Load the audio file
        audio, sr = librosa.load(audio_files[i], sr=sam_r)
        audio_segment = audio[s:e]

        # extract 5 MFCC coefficients from the selected audio segment and take transpose and then compute the mean
        mfcc_features = np.mean(librosa.feature.mfcc(y=audio_segment, sr=sam_r, n_mfcc=5).T, axis=0)
        MFCC.append(mfcc_features)         

        # extract 13 cepsrtal features from the selected audio segment then compute the mean
        plp_features = np.mean(plp(audio_segment, sam_r, 13), axis=0)
        PLP.append(plp_features)  

        # extract the speaker name as the label against the data entry
        speaker = classes[speaker]
        labels.append(speaker)


In [ ]:
# Save MFCC and PLP features and labels to numpy files
np.save('C:\\Users\\ApriZon\\Desktop\\CEP\\MFCC_features.npy', MFCC)
np.save('C:\\Users\\ApriZon\\Desktop\\CEP\\PLP_features.npy', PLP)
np.save('C:\\Users\\ApriZon\\Desktop\\CEP\\labels.npy', labels)

In [3]:
# 80% Training Data and 20% Test Data
# Splitting the data into training and test sets
X_train_mfcc, X_test_mfcc, y_train_mfcc, y_test_mfcc = train_test_split(MFCC, labels, test_size=0.2)
X_train_plp, X_test_plp, y_train_plp, y_test_plp = train_test_split(PLP, labels, test_size=0.2)

In [ ]:
# Train the GMM-UBM classifier
clf_gmm_mfcc = GaussianMixture(n_components=8, covariance_type='full', max_iter=500).fit(X_train_mfcc)

# Unsupervised Learning 
# Predict labels for the test set
y_pred_mfcc = clf_gmm_mfcc.predict(X_test_mfcc)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_mfcc, y_pred_mfcc)
print("Accuracy:", accuracy*100)

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],          # Regularization parameter
    'kernel': ['linear', 'rbf'],     # Kernel type
    'gamma': ['scale', 'auto']       # Kernel coefficient
}

# Create the SVM model
svm_model_mfcc = svm.SVC()

# Perform grid search using cross-validation
grid_search = GridSearchCV(svm_model_mfcc, param_grid, cv=5)
grid_search.fit(X_train_mfcc, y_train_mfcc)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test_mfcc, y_test_mfcc)
print("Accuracy with Best Hyperparameters:", accuracy*100)

In [4]:
clf_svm_mfcc = svm.SVC(C=10, kernel='linear', gamma='scale').fit(X_train_mfcc, y_train_mfcc)

# Predict labels for the test set
y_pred_mfcc = clf_svm_mfcc.predict(X_test_mfcc)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_mfcc, y_pred_mfcc)
print("Accuracy:", accuracy*100)

Accuracy: 84.14239482200647


In [ ]:
# Train the GMM-UBM classifier
clf_gmm_plp = GaussianMixture(n_components=8, covariance_type='full', max_iter=2000).fit(X_train_plp)

# Unsupervised Learning 
# Predict labels for the test set
y_pred_plp = clf_gmm_plp.predict(X_test_plp)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_plp, y_pred_plp)
print("Accuracy:", accuracy*100)

In [ ]:
clf_svm_plp = svm.SVC(C=100, kernel='linear', gamma='scale').fit(X_train_plp, y_train_plp)

# Predict labels for the test set
y_pred_plp = clf_svm_plp.predict(X_test_plp)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_plp, y_pred_plp)
print("Accuracy:", accuracy*100)

In [8]:
duration = 5  # Duration for recording in seconds

# Define the threshold for speaker identification
threshold = 0.6

# Load the dictionary mapping class labels to speaker identities
classes = {"ali":1,"ayesha":2,"mahnoor":3,"maryam":4,"usman_amin":5,"askari":6,"usman_hassan":7,"huzaifa":8}

testing_svm = svm.SVC(C=100, kernel='linear', gamma='scale').fit(MFCC, labels)

# Function to extract MFCC features from audio
def get_key(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

def extract_mfcc(audio, sam_r):
    mfccs = np.mean(librosa.feature.mfcc(y=audio.flatten(), sr=sam_r, n_mfcc=5).T, axis = 0)
    mfcc_features = mfccs.reshape(1,-1)
    return mfcc_features

# Function to identify the speaker from MFCC features
def identify_speaker(features):
    predicted_label = testing_svm.predict(features)[0]

    print('The predicted label is ' + get_key(classes,predicted_label))
    
    confidence = np.max(testing_svm.decision_function(features))
    
    if confidence > threshold:
        speaker = get_key(classes,predicted_label)
    else:
        speaker = "Unknown"
    
    return speaker

# Record audio in real-time and perform speaker identification
def real_time_speaker_identification():

    true_speaker = input('Enter the id of the speaker to be identified: ')

    # Open an audio stream for recording
    stream = sd.InputStream(samplerate=sam_r, channels=1)

    print("Recording has been started. Start Speaking in the mic...")
    stream.start()
    audio_frames = []

    # Record audio for the specified duration
    for _ in range(int(duration * sam_r)):
        audio_frames.append(stream.read(1)[0])

    # Stop the audio stream
    stream.stop()
    stream.close()

    print("Recording stopped. Processing on the Input Audio")

    # Convert the audio frames to a numpy array
    audio = np.array(audio_frames)

    # Save the recorded audio to a WAV file (optional)
    write("C:\\Users\\ApriZon\\Desktop\\CEP\\recorded_audio.wav", sam_r, audio)

    # Extract MFCC features from the recorded audio
    mfcc_features = extract_mfcc(audio, sam_r)

    # Identify the speaker from the MFCC features
    speaker = identify_speaker(mfcc_features)

    if speaker == true_speaker:
        print('System has successfully recognized the speaker from the database')
    else:
        print('Failed to Identify')

# Run the real-time speaker identification
real_time_speaker_identification()

Recording has been started. Start Speaking in the mic...
Recording stopped. Processing on the Input Audio
The predicted label is usman_amin
Failed to Identify
